Exercise 3.1

In [27]:
from importlib.metadata import version
import torch

In [28]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89],
   [0.55, 0.87, 0.66],
   [0.57, 0.85, 0.64],
   [0.22, 0.58, 0.33],
   [0.77, 0.25, 0.10],
   [0.05, 0.80, 0.55]]
)

d_in, d_out = 3, 2

In [29]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.randn(d_in, d_out))
        self.W_key = nn.Parameter(torch.randn(d_in, d_out))
        self.W_value = nn.Parameter(torch.randn(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attn_scores = queries @ keys.t()
        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)

In [30]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=False)
        self.W_key   = nn.Linear(d_in, d_out, bias=False)
        self.W_value = nn.Linear(d_in, d_out, bias=False)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v2 = SelfAttention_v2(d_in, d_out)

In [31]:
sa_v1.W_query = torch.nn.Parameter(sa_v2.W_query.weight.T)
sa_v1.W_key = torch.nn.Parameter(sa_v2.W_key.weight.T)
sa_v1.W_value = torch.nn.Parameter(sa_v2.W_value.weight.T)

In [37]:
sa_v1(inputs)
sa_v2(inputs)

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)

Exercise 3.2

In [33]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length),
            diagonal=1)
        )
    def forward(self, x):
        b, num_tokens, d_in = x.shape #3
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_( #4
        self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(
        attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values
        return context_vec

In [34]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(
                d_in, d_out, context_length, dropout, qkv_bias
            )
            for _ in range(num_heads)]
        )
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [36]:
batch = torch.stack((inputs, inputs), dim=0)
torch.manual_seed(123)
d_out = 1
context_length = batch.shape[1]
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vec = mha(batch)
print(context_vec)
print(context_vec.shape)

tensor([[[-0.5740,  0.2216],
         [-0.7320,  0.0155],
         [-0.7774, -0.0546],
         [-0.6979, -0.0817],
         [-0.6538, -0.0957],
         [-0.6424, -0.1065]],

        [[-0.5740,  0.2216],
         [-0.7320,  0.0155],
         [-0.7774, -0.0546],
         [-0.6979, -0.0817],
         [-0.6538, -0.0957],
         [-0.6424, -0.1065]]], grad_fn=<CatBackward0>)
torch.Size([2, 6, 2])


Exercise 3.3


In [40]:
torch.manual_seed(123)
d_in, d_out = 768,768
context_length = 1024
num_heads = 12
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads)

batch_size = 2
num_tokens = 6
new_batch = torch.randn(batch_size, num_tokens, d_in)
context_vec = mha(new_batch)
print(context_vec)
print(context_vec.shape)

tensor([[[ 0.0658,  0.4906,  0.3642,  ...,  0.1755, -0.1827,  0.0292],
         [-0.0190, -0.5791, -0.1285,  ..., -0.5557, -0.4462, -0.0123],
         [-0.1720, -0.2992, -0.0185,  ..., -0.2741, -0.3955, -0.2417],
         [ 0.1284,  0.0102,  0.2645,  ..., -0.0180, -0.2341, -0.2064],
         [ 0.1278,  0.1428,  0.1062,  ...,  0.0486, -0.1233, -0.1398],
         [ 0.0964,  0.1203,  0.1756,  ..., -0.0337, -0.3025, -0.2235]],

        [[-0.7179,  0.3238, -0.3076,  ...,  0.9910,  0.1504, -0.2514],
         [ 0.3219,  0.3941,  0.2757,  ...,  0.5721,  0.4248, -0.3072],
         [-0.1234,  0.0817,  0.3731,  ...,  0.2150,  0.3889, -0.5661],
         [-0.0953,  0.1454,  0.0174,  ..., -0.0400,  0.2972, -0.3992],
         [-0.1681,  0.0595, -0.0884,  ..., -0.0955,  0.0814, -0.3630],
         [-0.0532,  0.1850,  0.1957,  ..., -0.1563,  0.0554, -0.1444]]],
       grad_fn=<CatBackward0>)
torch.Size([2, 6, 9216])


In [41]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(mha))

21233664
